In [ ]:
from tqdm import tqdm
import pandas as pd
import torchaudio
import librosa
import shutil
import json
import os

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=8, progress_bar=True)

In [ ]:
def load_data(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.readlines()
        lines = [json.loads(line.strip()) for line in content]
    data = pd.DataFrame(lines)

    return data

In [ ]:
in_jsonl_path = "../data/metadata/raw-jsonl/train-data-type-9.jsonl"
out_jsonl_path = "../data/metadata/filtered-jsonl/train-data-type-9.jsonl"

data = load_data(in_jsonl_path)
print(data.shape)
data.head(2)

In [ ]:
scores = data.apply(lambda x: x["phone_scores"], axis=1).to_list()
scores = [score for sample in scores for score in sample]

pd.DataFrame(scores, columns=["score"]).score.hist(bins=100)

In [ ]:
filtered_data = []
for name, group in data.groupby("text"):
    if group.shape[0] >= 268:
        samples = group.sample(268)
        filtered_data.append(samples)
    else:
        filtered_data.append(group)

filtered_data = pd.concat(filtered_data)
filtered_data.shape

In [ ]:
scores = filtered_data.apply(lambda x: x["phone_scores"], axis=1).to_list()
scores = [score for sample in scores for score in sample]

pd.DataFrame(scores, columns=["score"]).score.hist(bins=100)

In [ ]:
filtered_data.utterance_score.hist(bins=100)

In [ ]:
scores = filtered_data.apply(lambda x: x["word_scores"], axis=1).to_list()
scores = [score for sample in scores for score in sample]

pd.DataFrame(scores, columns=["score"]).score.hist(bins=100)

In [ ]:
filtered_data.text.value_counts().hist()

In [ ]:
from tqdm import tqdm
 
with open(out_jsonl_path, "w", encoding="utf-8") as f:
    for index in tqdm(filtered_data.index):
        sample = filtered_data.loc[index].to_dict()
        json_obj = json.dumps(sample)

        f.write(f'{json_obj}\n')